## Convert bam files to bigwig files

In [ ]:
sos run /mnt/vast/hpc/homes/rf2872/codes/xqtl-pipeline/code/misc/bam_to_bw.ipynb reformat_vcf_wasp \
    --bam_files `ls /mnt/vast/hpc/csg/rf2872/data/data_ROSMAP/test_bams/*Aligned.sortedByCoord.out.md.bam` \
    --container /mnt/vast/hpc/csg/rf2872/Work/test/rna_quantification.sif

In [ ]:
[global]
import os
# Work directory & output directory
parameter: cwd = path('./')
# The filename prefix for output data
parameter: job_size = 1
parameter: mem = '60G'
parameter: container = ""
parameter: entrypoint={('micromamba run -a "" -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}
parameter: bam_files = paths
import pandas as pd
#parameter: analysis_units = path
# handle N = per_chunk data-set in one job
parameter: per_chunk = 1


In [ ]:
[reformat_vcf_wasp]
input: bam_files, group_by = 1
output:f'{cwd:a}/{_input:bn}.bw'
task: trunk_workers = 1, trunk_size = job_size, mem = mem,  walltime = '24h', tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container=container, entrypoint=entrypoint
    samtools index $[_input]
    bamCoverage -b $[_input] -o $[_output]
